In [1]:
import sys
import os
import pandas as pd
import numpy as np
import talib as ta
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
import matplotlib.pyplot as plt
import datetime
import random

from pypfopt import expected_returns, risk_models
from pypfopt import EfficientFrontier
from pypfopt import HRPOpt

In [2]:
# dataset = pd.read_csv('./IndexData.csv').set_index('Date')
dataset = pd.read_excel('./Bloomberg_DRL.xlsx', sheet_name = 'Data',index_col='Date')
dataset['UST_JPY'] = dataset['SPBDU10T Index'] * dataset['JPY BGN Curncy']
dataset['SPX_JPY'] = dataset['SPXT Index'] * dataset['JPY BGN Curncy']

In [3]:
target_name = ['UST_JPY', 'SPX_JPY', 'TPXDDVD Index', 'SPJGBTR Index']
feature_name = dataset.columns.drop(target_name)

In [4]:
dataset.head()

,SPX Index,SX5E Index,TPX Index,MXEF Index,SHCOMP Index,HSI Index,ASX Index,JPY BGN Curncy,EUR BGN Curncy,GBP BGN Curncy,...,IMM3CNCN Index,CESIEM Index,CESIJPY Index,CESIGL Index,SPBDU10T Index,SPXT Index,TPXDDVD Index,SPJGBTR Index,UST_JPY,SPX_JPY
Date,,,,,,,,,,,,,,,,,,,,,
Code,1.00,1.00,1.00,1.00,1.000,1.00,1.00,1.00,1.0000,1.0000,...,3,3.0,3.0,3.0,1.000,1.00,1.00,1.00,1.00000,1.0000
2003-01-03 00:00:00,908.59,2502.19,843.29,299.98,1319.868,9583.85,1923.75,119.63,1.0428,1.6113,...,11519,20.2,-11.8,4.4,296.102,1302.55,942.45,124.04,35422.68226,155824.0565
2003-01-10 00:00:00,927.57,2489.01,837.70,302.86,1384.859,9721.50,1911.39,119.32,1.0576,1.6076,...,13131,19.3,-4.2,3.6,294.035,1330.29,936.20,123.82,35084.25620,158730.2028
2003-01-17 00:00:00,901.78,2390.36,859.25,301.96,1478.675,9614.59,1844.77,117.83,1.0678,1.6131,...,24982,15.4,-6.5,2.3,296.949,1293.63,960.29,124.34,34989.50067,152428.4229
2003-01-24 00:00:00,861.40,2234.56,861.36,293.90,1479.070,9460.60,1749.49,117.82,1.0827,1.6333,...,31985,10.8,-13.6,9.1,299.355,1235.84,962.65,124.61,35270.00610,145606.6688


In [5]:
window = int(52 * 4) # rolling window
StartDate = pd.to_datetime(datetime.datetime(2006, 12, 29)) - pd.offsets.Week(window)
EndDate = pd.to_datetime(datetime.datetime(2019, 12, 31)) 
df_data = dataset[target_name][1:][StartDate:EndDate]

In [6]:
df_data.head()

,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
Date,,,,
2003-01-03 00:00:00,35422.68226,155824.0565,942.45,124.04
2003-01-10 00:00:00,35084.25620,158730.2028,936.20,123.82
2003-01-17 00:00:00,34989.50067,152428.4229,960.29,124.34
2003-01-24 00:00:00,35270.00610,145606.6688,962.65,124.61
2003-01-31 00:00:00,35739.12063,147217.1418,917.81,124.68


In [7]:
mu = expected_returns.mean_historical_return(df_data, frequency=52)
S = risk_models.sample_cov(df_data)
ef = EfficientFrontier(mu, S)
ef.min_volatility()

{'UST_JPY': 0.0207623584738611,
 'SPX_JPY': 0.0098283559397343,
 'TPXDDVD Index': 0.0503886724724232,
 'SPJGBTR Index': 0.9190206131139814}

In [8]:
ef.efficient_return(target_return=0.05)

{'UST_JPY': 0.2240914916058734,
 'SPX_JPY': 0.249954384689466,
 'TPXDDVD Index': 0.0509036265134478,
 'SPJGBTR Index': 0.4750504971912127}

In [9]:
print(mu)
S

UST_JPY          0.044351
SPX_JPY          0.112616
TPXDDVD Index    0.079773
SPJGBTR Index    0.016528
dtype: float64


,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
UST_JPY,0.036083,0.036712,0.024342,0.000393
SPX_JPY,0.036712,0.218207,0.130857,-0.007402
TPXDDVD Index,0.024342,0.130857,0.183489,-0.009074
SPJGBTR Index,0.000393,-0.007402,-0.009074,0.003504


In [10]:
df_data.pct_change().mean(axis=0)*52

UST_JPY          0.044351
SPX_JPY          0.112616
TPXDDVD Index    0.079773
SPJGBTR Index    0.016528
dtype: float64

In [11]:
ef.max_sharpe(risk_free_rate=0.01)

{'UST_JPY': 0.0913317595611203,
 'SPX_JPY': 0.093168848142239,
 'TPXDDVD Index': 0.0505673974672275,
 'SPJGBTR Index': 0.764931994829413}

In [12]:
# minimum variance
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.min_volatility())

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_GMV.csv')

In [13]:
# maximize sharpe ratio
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.max_sharpe(risk_free_rate=0.001))

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_MaxSharpe.csv')

In [14]:
# mean variance
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.max_quadratic_utility(risk_aversion=3.0))

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_MV.csv')

In [15]:
df_data_pct = df_data.pct_change().dropna()
df_data_pct.head()

,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
Date,,,,
2003-01-10 00:00:00,-0.009554,0.018650,-0.006632,-0.001774
2003-01-17 00:00:00,-0.002701,-0.039701,0.025732,0.004200
2003-01-24 00:00:00,0.008017,-0.044754,0.002458,0.002171
2003-01-31 00:00:00,0.013301,0.011060,-0.046580,0.000562
2003-02-07 00:00:00,0.007096,-0.026629,0.021824,-0.001123


In [16]:
# hierarchical risk parity
weight = []
for w in range(len(df_data)-window-1): # 時点注意(0スタート)
    returns = df_data_pct[w:w+window]
    hrp = HRPOpt(returns)
    weight.append(hrp.optimize())

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df = weight_df.loc[:, target_name]
weight_df.to_csv('weight_HRP.csv')

In [17]:
# original target return strategy
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    # print(w, mu, df_data.index[w+window])
    # ef = EfficientFrontier(mu, S, weight_bounds=[(0, 0.5), (0, 0.5), (0, 0.5), (0, 1.0)])
    ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.999))
    try:
        weight.append(ef.efficient_return(0.05, market_neutral=False))
    except:
        try:
            weight.append(ef.efficient_return(0.03, market_neutral=False))
            print('0.03', df_data.index[w+window])
        except:
            try:
                weight.append(ef.efficient_return(0.01, market_neutral=False))
                print('0.01', df_data.index[w+window])
            except:
                weight.append(ef.efficient_return(0.0001, market_neutral=False))
                print('0.0001', df_data.index[w+window])
    # except OptimizationError:

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_TargetReturn.csv')

0.03 2008-10-03 00:00:00
0.03 2008-10-10 00:00:00
0.03 2008-10-17 00:00:00
0.03 2008-10-24 00:00:00
0.01 2008-10-31 00:00:00
0.03 2008-11-07 00:00:00
0.03 2008-11-14 00:00:00
0.03 2008-11-21 00:00:00
0.03 2008-12-05 00:00:00
0.03 2008-12-12 00:00:00
0.03 2008-12-19 00:00:00
0.03 2009-01-16 00:00:00
0.03 2009-01-23 00:00:00
0.03 2009-01-30 00:00:00
0.03 2009-02-06 00:00:00
0.03 2009-02-13 00:00:00
0.03 2009-02-20 00:00:00
0.03 2009-02-27 00:00:00
0.03 2009-05-08 00:00:00
0.03 2009-05-15 00:00:00
0.03 2009-05-22 00:00:00
0.01 2009-05-29 00:00:00
0.01 2009-06-05 00:00:00
0.03 2009-06-12 00:00:00
0.01 2009-06-19 00:00:00
0.01 2009-06-26 00:00:00
0.01 2009-07-03 00:00:00
0.01 2009-07-10 00:00:00
0.01 2009-07-17 00:00:00
0.01 2009-07-24 00:00:00
0.01 2009-07-31 00:00:00
0.01 2009-08-07 00:00:00
0.01 2009-08-14 00:00:00
0.01 2009-08-21 00:00:00
0.01 2009-08-28 00:00:00
0.01 2009-09-04 00:00:00
0.01 2009-09-11 00:00:00
0.01 2009-09-18 00:00:00
0.01 2009-09-25 00:00:00
0.01 2009-10-02 00:00:00


In [20]:
# Risk Parity without risk budgeting
from pyrb import EqualRiskContribution, RiskBudgeting, RiskBudgetAllocation

weight = []
for w in range(len(df_data)-window-1): # 時点注意(0スタート)
    returns = df_data_pct[w:w+window]
    covariance_matrix = returns.cov() * 52
    ERC = EqualRiskContribution(covariance_matrix)
    ERC.solve()
    weight.append(ERC.x)
weight_df = pd.DataFrame(weight, columns=list(df_data.columns))
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_RP.csv')

In [23]:
# Risk Parity with risk budgeting
weight = []
for w in range(len(df_data)-window-1): # 時点注意(0スタート)
    returns = df_data_pct[w:w+window]
    covariance_matrix = returns.cov() * 52
    budgets = [0.07, 0.5, 0.4, 0.03] # 適当に設定したBudget 
    RB = RiskBudgeting(covariance_matrix,budgets)
    RB.solve()
    weight.append(RB.x)
weight_df = pd.DataFrame(weight, columns=list(df_data.columns))
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_RPwithBudget.csv')